In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell'


In [2]:
kafka_broker_hostname = 'kafka-49b7861-rafaelathaydemello-3b01.aivencloud.com'
kafka_broker_port = '25697'
kafka_broker = kafka_broker_hostname + ':' +kafka_broker_port
kafka_topic= 'picture-metadata'

spark = SparkSession.builder.appName('PicturesAnalysis').getOrCreate()


In [75]:
    

# Batch Query
df = spark.read.format('kafka') \
    .option('kafka.bootstrap.servers', kafka_broker)\
    .option("kafka.security.protocol","SSL")\
    .option("kafka.ssl.endpoint.identification.algorithm", "")\
    .option("kafka.ssl.truststore.location", "/home/jovyan/local/client.truststore.jks")\
    .option("kafka.ssl.truststore.password", "password")\
    .option("kafka.ssl.keystore.type", "PKCS12")\
    .option('kafka.ssl.keystore.location', "/home/jovyan/local/client.keystore.p12")\
    .option('kafka.ssl.keystore.password', "password")\
    .option('subscribe', kafka_topic)\
    .option('startingOffsets', 'earliest')\
    .option("includeHeaders", "true")\
    .load()


In [76]:
# .option("sslmode", "require" ) 

In [77]:
df.show()

+--------------------+--------------------+----------------+---------+------+--------------------+-------------+-------+
|                 key|               value|           topic|partition|offset|           timestamp|timestampType|headers|
+--------------------+--------------------+----------------+---------+------+--------------------+-------------+-------+
|[32 30 32 30 2D 3...|[7B 22 73 75 63 6...|picture-metadata|        0|     0|2020-08-09 04:04:...|            0|   null|
|[32 30 32 30 2D 3...|[7B 22 73 75 63 6...|picture-metadata|        0|     1|2020-08-09 04:04:...|            0|   null|
|[32 30 32 30 2D 3...|[7B 22 73 75 63 6...|picture-metadata|        0|     2|2020-08-09 04:04:...|            0|   null|
|[32 30 32 30 2D 3...|[7B 22 73 75 63 6...|picture-metadata|        0|     3|2020-08-09 04:04:...|            0|   null|
|[32 30 32 30 2D 3...|[7B 22 73 75 63 6...|picture-metadata|        0|     4|2020-08-09 04:04:...|            0|   null|
|[32 30 32 30 2D 3...|[7B 22 73 

In [7]:
# Streaming Query
df = spark.readStream.format('kafka') \
    .option('kafka.bootstrap.servers', kafka_broker)\
    .option('sasl.cafile', "ca.pem")\
    .option('ssl.certfile', 'service.cert')\
    .option('ssl.keyfile', 'service.key')\
    .option('subscribe', kafka_topic)\
    .option("includeHeaders", "true")\
    .load()

In [8]:
df.writeStream.format("memory").queryName("hello").start()
raw = spark.sql("select * from hello")
raw.show

AnalysisException: Queries with streaming sources must be executed with writeStream.start();;
kafka